# Proyecto Modelos I - UdeA
## Tercer Entrega

En este notebook se desarrolla la fase final del proyecto del curso “Modelos y Simulación de Sistemas I” de la Universidad de Antioquia.
A partir del conjunto de datos preprocesado en la entrega anterior, se construye, entrena y evalúa un modelo de aprendizaje automático que permita predecir el rendimiento global de los estudiantes, en el contexto de la competencia “Pruebas Saber Pro Colombia”.

El propósito de esta entrega es implementar un pipeline completo de modelado predictivo, iniciando desde la carga del dataset procesado y finalizando con la generación del archivo submission.csv requerido para la plataforma Kaggle.

<br>
---

# Antes de empezar
Es necesario importar y descargar los archivos requeridos desde la plataforma Kaggle. Para ello, se cargan primero las librerías necesarias en el notebook y, posteriormente, se descarga el archivo comprimido (.zip) que contiene el dataset principal (train.csv), el cual será utilizado en el proceso de preprocesado para luego realizar las predicciones.

In [ ]:
# Instalamos la librería necesaria para el Target Encoding
!pip install category_encoders xgboost

import pandas as pd
import numpy as np
import unicodedata
import re
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score
from xgboost import XGBClassifier
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 3.1 MB/s eta 0:00:00
udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


## Descomprimimos el archivo .zip

In [ ]:
# Comando de linux para descomprimir sin mostrar nada en pantalla
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


## Cargamos los datos a analizar en un dataframe y verificamos que si haya cargado correctamente

También extraemos los ID del conjunto de prueba, ya que los necesitaremos al final para generar el archivo de submission.

In [ ]:
# Cargar datos
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
test_ids = df_test["ID"]


## Limpieza de datos - Columnas que tengan texto

Se procede a limpiar las columnas que contienen texto, eliminando tildes, mayúsculas y caracteres especiales.

Aquí definimos una función robusta `limpiar_texto` que:
1. Convierte todo a minúsculas.
2. Elimina tildes y acentos (normalización Unicode).
3. Elimina caracteres especiales (incluyendo guiones), dejando solo letras y números.

Aplicamos esta limpieza a todas las columnas de tipo texto (object) para reducir la cardinalidad y el ruido en los datos.

<b>Nota</b>: Omitimos la limpieza en la columna E_VALORMATRICULAUNIVERSIDAD ya que esta columna contiene datos con números separados por punto.


In [ ]:
def limpiar_texto(texto):
    if pd.isna(texto):
        return texto
    texto = str(texto)
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    texto = texto.lower()
    # Elimina todo lo que no sea letra o número
    texto = re.sub(r'[^a-z0-9\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto


# Aplicamos limpieza a todas las columnas de texto (menos la de valor matricula)
cols_texto = df.select_dtypes(include=['object']).columns.drop(["E_VALORMATRICULAUNIVERSIDAD"], errors='ignore')
cols_texto_test = df_test.select_dtypes(include=['object']).columns.drop(["E_VALORMATRICULAUNIVERSIDAD"], errors='ignore')

for col in cols_texto:
    df[col] = df[col].apply(limpiar_texto)
for col in cols_texto_test:
    df_test[col] = df_test[col].apply(limpiar_texto)

# Preparación del Target y Nulos

En esta sección realizamos tres pasos clave:
1. **Mapeo del Target:** Convertimos la variable objetivo `RENDIMIENTO_GLOBAL` a números. **Nota:** Como nuestra limpieza de texto eliminó los guiones, mapeamos "mediobajo" (sin guion) en lugar de "medio-bajo".
2. **Separación de Variables:** Definimos nuestra matriz de características `X` y el vector objetivo `y`.
3. **Imputación de Nulos:** Rellenamos los valores faltantes de forma sencilla pero efectiva: la mediana para columnas numéricas y la etiqueta "sin_dato" para categóricas.

In [ ]:
mapa_rendimiento = {
    "bajo": 0,
    "mediobajo": 1,
    "medioalto": 2,
    "alto": 3
}

# Separamos X e y
X = df.drop(columns=["RENDIMIENTO_GLOBAL", "ID"])
y = df["RENDIMIENTO_GLOBAL"].map(mapa_rendimiento)

# Verificamos que no haya nulos en el target
if y.isnull().sum() > 0:
    y = y.fillna(0)

X_test_submission = df_test.drop(columns=["ID"])

# Numéricas con la mediana
num_cols = X.select_dtypes(include=['number']).columns
X[num_cols] = X[num_cols].fillna(X[num_cols].median())
X_test_submission[num_cols] = X_test_submission[num_cols].fillna(X[num_cols].median())

# Categóricas con 'sin_dato'
cat_cols = X.select_dtypes(include=['object']).columns
X[cat_cols] = X[cat_cols].fillna('sin_dato')
X_test_submission[cat_cols] = X_test_submission[cat_cols].fillna('sin_dato')

# División de Datos y Codificación

Esta es la etapa más crítica para mejorar la precisión del modelo.

1. **Data Splitting:** Dividimos en entrenamiento (`train`) y validación (`valid`) **ANTES** de codificar.
2. **Target Encoding:** Aplicamos esta técnica a la columna `E_PRGM_ACADEMICO` (Carrera). En lugar de usar un simple número arbitrario, reemplazamos cada carrera por el **promedio de rendimiento** de los estudiantes de esa carrera. Esto le da al modelo una señal más potente.
3. **One-Hot Encoding:** Para el resto de variables categóricas con menos cardinalidad, usamos variables dummy (0 y 1).

In [ ]:
# Dividimos ANTES de codificar para evitar Data Leakage
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Target Encoding para la carrera específica
encoder = ce.TargetEncoder(cols=['E_PRGM_ACADEMICO'], smoothing=10)
encoder.fit(X_train, y_train)

X_train['E_PRGM_ACADEMICO_ENCODED'] = encoder.transform(X_train)['E_PRGM_ACADEMICO']
X_valid['E_PRGM_ACADEMICO_ENCODED'] = encoder.transform(X_valid)['E_PRGM_ACADEMICO']
X_test_submission['E_PRGM_ACADEMICO_ENCODED'] = encoder.transform(X_test_submission)['E_PRGM_ACADEMICO']

# Eliminamos la columna de texto original de la carrera
X_train.drop(columns=['E_PRGM_ACADEMICO'], inplace=True)
X_valid.drop(columns=['E_PRGM_ACADEMICO'], inplace=True)
X_test_submission.drop(columns=['E_PRGM_ACADEMICO'], inplace=True)

# One-Hot Encoding para el resto de categóricas
X_train = pd.get_dummies(X_train, drop_first=True)
X_valid = pd.get_dummies(X_valid, drop_first=True)
X_test_submission = pd.get_dummies(X_test_submission, drop_first=True)

# Alineación de columnas
cols_train = X_train.columns
X_valid = X_valid.reindex(columns=cols_train, fill_value=0)
X_test_submission = X_test_submission.reindex(columns=cols_train, fill_value=0)

# Entrenamiento del Modelo (XGBoost)

Utilizamos **XGBoost**, uno de los algoritmos más potentes para datos tabulares. Hemos ajustado los hiperparámetros para buscar un equilibrio entre precisión y generalización:

* `n_estimators=800`: Un alto número de árboles para aprender patrones sutiles.
* `learning_rate=0.02`: Un aprendizaje lento para evitar sobreajustarse rápido.
* `max_depth=8`: Árboles lo suficientemente profundos para capturar interacciones complejas.
* `tree_method='hist'`: Optimización para acelerar el entrenamiento.

In [ ]:
model = XGBClassifier(
    n_estimators=800,        # Muchos árboles para aprender bien
    learning_rate=0.02,      # Aprendizaje lento para mayor precisión
    max_depth=8,             # Profundidad media-alta
    subsample=0.8,           # Evita sobreajuste usando 80% de datos por árbol
    colsample_bytree=0.8,    # Evita sobreajuste usando 80% de columnas
    objective="multi:softprob",
    num_class=4,
    random_state=42,
    tree_method="hist",      # Aceleración por histogramas
    n_jobs=-1                # Usar todos los núcleos
)

model.fit(X_train, y_train, verbose=False)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.02, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=800,
              n_jobs=-1, num_class=4, ...)

# Evaluación del Rendimiento

Evaluamos qué tan bueno es nuestro modelo usando el conjunto de validación (datos que el modelo nunca vio durante el entrenamiento).

Analizamos:
* **Accuracy:** El porcentaje total de aciertos.
* **Classification Report:** Para ver la precisión y sensibilidad (recall) en cada una de las 4 clases de rendimiento (bajo, medio-bajo, medio-alto, alto).

In [ ]:
y_pred_valid = model.predict(X_valid)
acc = accuracy_score(y_valid, y_pred_valid)
f1 = f1_score(y_valid, y_pred_valid, average='macro')

print(f"\n RESULTADOS EN VALIDACIÓN:")
print(f"Accuracy: {acc:.4f}")
print(f"F1 Score (Macro): {f1:.4f}")
print("\nReporte de Clasificación:")
print(classification_report(y_valid, y_pred_valid))


 RESULTADOS EN VALIDACIÓN:
Accuracy: 0.4358
F1 Score (Macro): 0.4241

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.46      0.57      0.51     34597
           1       0.33      0.28      0.30     34455
           2       0.33      0.27      0.30     34324
           3       0.55      0.62      0.59     35124

    accuracy                           0.44    138500
   macro avg       0.42      0.43      0.42    138500
weighted avg       0.42      0.44      0.43    138500



# Predicción Final y Generación de Submission

Finalmente, aplicamos el modelo entrenado al conjunto de prueba (`test.csv`).

Es crucial realizar el **Mapeo Inverso**: convertimos las predicciones numéricas (0, 1, 2, 3) de nuevo a texto ("bajo", "medio-bajo"...). Aquí **reintroducimos los guiones**, ya que se espera el formato exacto "medio-bajo" y "medio-alto".

Generamos el archivo `submission_final.csv`.

In [ ]:
pred_test_final = model.predict(X_test_submission)

# Mapeo inverso: Convertimos números a texto CON GUIONES (formato original)
mapa_inverso = {
    0: "bajo",
    1: "medio-bajo",
    2: "medio-alto",
    3: "alto"
}

pred_texto = pd.Series(pred_test_final).map(mapa_inverso)

submission = pd.DataFrame({
    "ID": test_ids,
    "RENDIMIENTO_GLOBAL": pred_texto
})

submission.to_csv("submission.csv", index=False)

## Conclusión

En este notebook se completó la etapa de predicción de los datos.

Este proceso refleja la importancia de combinar un buen preprocesamiento, una correcta elección de algoritmos y una validación rigurosa para construir modelos robustos.